In [ ]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../code'))
import config
from spellcheck import Spellcheck
from util import Util

# hny_pap_nl = pd.read_csv("../data/hny/pap-nl.csv", na_filter=False)
nbo_pap = pd.read_csv("../data/nbo/pap.csv", na_filter=False)
corpus = Util.attachType(nbo_pap, "pap-simple")
corpus = corpus[corpus["type"]=="word"]
words = corpus["pap-simple"].values
spell = Spellcheck(spellchecker_corpus=words)

sentence = "mi man no ta bon pero manan mi kuna ta trese anto bini bek"
big_sentence = ". Presidente di Banko Sentral di Kòrsou i St. Maarten sr. Richard Doornbosch... WILLEMSTAD.- Buskando solushon fo’i 2013 te ku 2019 pa Girobank, a hasi investigashon di e banko. A aserka APC i IIG TOF Holding N.V. (ámbos ta doño di Girobank ku respektivamente 42% i 58%) pa un plan di restrukturashon i re-kapitalisashon pa tin sufisiente kapital den e banko; identifiká potensial kumpradónan ku ke a invertí den e banko. Esaki no tabata algu fásil. Tabatin un défisit"

In [ ]:
# Translate.getWordCorrections(sentence, corpus)

In [ ]:
# trans.getFastWordCorrections(sentence, check_alternatives=True)

In [ ]:
Translate.distanceToWord("Kòrsou", "Kòrsou")
tst = Translate.attachClosest(corpus, "Kòrsou", "pap-simple")
print(tst.head(3))
if tst["closest"].iloc[0] > 0:
    print("ask")

In [ ]:
import nltk
# trans.getMixedWordCorrections(sentence, corpus)
smarter = "iasabra el a bisami ekos kune i use"
input_words = Util.findWords(big_sentence)
# print(input_words)

In [ ]:
spell.getMixedWordCorrections(input_words, corpus, "Levenstein")

In [ ]:
spell.getMixedWordCorrections(input_words, corpus, metric="Hamming")

In [ ]:
spell.getWordCorrections(input_words, metric="Levenstein")

In [ ]:
# characters_test = "(whatever)i don't know, but random. 15 a)e right {,age}()((er)"
characters_test = "min sa 42% 32 fo'i ki dia re-kapitulashon M'a mire n' ta nothing' bai fo'i 'esaki tambe' 'gani sa?' e"
# characters_test = "Buskando solushon fo’i 2013 te ku 2019"
import re
def getWords(sentence):
    # words = re.findall(r'^(.*?)[;\.,!\(].*',without_num)        
    # words = re.findall(r'[^\w\d\'](\w+)[^\w\d\']',sentence) 
    # words = re.findall(r'\b([^\d\W]+[\'-]?[^\d\W\']*)\b',sentence) 
    words = re.findall(r'\b([^\d\W]+[\'\’-]*[^\d\W\']*)',sentence) 
    # words = re.findall(r'\b([\d\w\'-]+)\b',sentence) 
    # words = re.findall(r'([a-z]+[\'-]?(?![\w\'-])[a-z]*)',sentence) 
    return words
getWords(characters_test)
# getWords(big_sentence)